In [ ]:
!which chromedriver

In [1]:
import pymongo
from bs4 import BeautifulSoup as bs
import os
import requests
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import urllib.request
import urllib.parse
import re

In [2]:
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", usr/local/bin/chromedriver, headless=False)

### NASA Mars News

In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.NASA_Mars_news_db
stories = db.stories

In [5]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<!-- Always force latest IE rendering engine or request Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<!-- Responsiveness -->
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<!-- Favicon -->
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/manifest.json" rel="manifest"/>
<link color="#e48b55" href="/safari-pinned-tab.svg" rel="mask-icon"/>
<meta content="#000000" name="theme-color"/>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="N60xQlc7wHhKtKgP2euzLu4dB6Xuj0Z3WYJ2oQQZeuxj8gHaNsuHWpZ2ApOJVZNWDjtl38vjd709lBidee6Ilg==" name="csrf-t

In [6]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
#<div class="slide">
#links = [a['href'] for a in soup.find_all('div' class_="slide"]
#img_url = [a['href'] for a in soup.select('a:contains(Original)')]


#links = [a['href'] for a in soup.select('a:contains(news)')]

#links = soup.select("a[href*=news]")
#links = [a['href'] for a in soup.find_all('a', href=True)]

#Works, but gets a couple of extras and others twice:
links = [a['href'] for a in soup.select("a[href*=news]")]


#links = [a['href*=news'] for a in soup.select("div[class=slide]")]
links

['/news/8749/nasa-readies-perseverance-mars-rovers-earthly-twin/',
 '/news/8749/nasa-readies-perseverance-mars-rovers-earthly-twin/',
 '/news/8716/nasa-to-broadcast-mars-2020-perseverance-launch-prelaunch-activities/',
 '/news/8716/nasa-to-broadcast-mars-2020-perseverance-launch-prelaunch-activities/',
 '/news/8695/the-launch-is-approaching-for-nasas-next-mars-rover-perseverance/',
 '/news/8695/the-launch-is-approaching-for-nasas-next-mars-rover-perseverance/',
 '/news/8692/nasa-to-hold-mars-2020-perseverance-rover-launch-briefing/',
 '/news/8692/nasa-to-hold-mars-2020-perseverance-rover-launch-briefing/',
 '/news/8659/alabama-high-school-student-names-nasas-mars-helicopter/',
 '/news/8659/alabama-high-school-student-names-nasas-mars-helicopter/',
 '/news/8645/mars-helicopter-attached-to-nasas-perseverance-rover/',
 '/news/8645/mars-helicopter-attached-to-nasas-perseverance-rover/',
 '/#news_and_events',
 '/news']

In [69]:
# Loop through returned results
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
base = "https://mars.nasa.gov/news"
used_list = []
url_list = []
for link in links:
    if link not in used_list:    
        url = urllib.parse.urljoin(base,link)
        url_list.append(url) 
        used_list.append(link)
#url
#url_list
for url in url_list:
    browser.visit(url)
    try:
#        news_title = browser.find_by_tag('h1')
#        news_p = browser.find_by_id('primary_column')
        news_title = browser.find_by_tag('h1').value
        news_p = browser.find_by_id('primary_column').value
        news_p = news_p.replace('\n','')
        news_p = news_p.replace('\'s ', '&')
        news_p = news_p.replace('&', "'s ")
        #        print ('-------------')
#        print (news_title)
#        print (news_p)
        # Dictionary to be inserted as a MongoDB document
        post = {
            'news_title': news_title,
#            'news_p': news_p.replace('\n', '')
            'news_p': news_p          
        }
        stories.insert_one(post)
        print(post)

    except Exception as e:
        print(e)

browser.quit()        
#print(stories)
#row.replace('\n', '')

{'news_title': "NASA Readies Perseverance Mars Rover's Earthly Twin", 'news_p': 'Did you know NASA\'s next Mars rover has a nearly identical sibling on Earth for testing? Even better, it\'s about to roll for the first time through a replica Martian landscape.As NASA\'s Mars rover Perseverance hurtles through space toward the Red Planet, the six-wheeler\'s twin is ready to roll here on Earth.A full-scale engineering version of the Mars 2020 Perseverance rover – outfitted with wheels, cameras, and powerful computers to help it drive autonomously – has just moved into its garage home at NASA\'s Jet Propulsion Laboratory in Southern California. This rover model passed its first driving test in a relatively tame warehouselike assembly room at JPL on Sept. 1. Engineers expect to take it out next week into the Mars Yard, where a field of red dirt studded with rocks and other obstacles simulates the Red Planet\'s surface.Download video ›"Perseverance\'s mobility team can\'t wait to finally dri

{'news_title': "The Launch Is Approaching for NASA's Next Mars Rover, Perseverance", 'news_p': 'The Red Planet\'s surface has been visited by eight NASA spacecraft. The ninth will be the first that includes gathering Mars samples for future return to Earth.NASA\'s Perseverance Mars rover is just over a month from its July 20 targeted launch date. The rover\'s astrobiology mission will seek signs of past microscopic life on Mars, explore the geology of the Jezero Crater landing site, and demonstrate key technologies to help prepare for future robotic and human exploration. And the rover will do all that while collecting the first samples of Martian rock and regolith (broken rock and dust) for return to Earth by a set of future missions."Fifty-one years ago today, NASA was deep into final preparations for the first Moon landing," said NASA administrator Jim Bridenstine. "Today we stand at the threshold of another monumental moment in exploration: sample collection at Mars. As we celebrat

{'news_title': "Alabama High School Student Names NASA's Mars Helicopter", 'news_p': 'High School Junior Names NASA\'s Mars Helicopter: Vaneeza Rupani (inset), a junior at Tuscaloosa County High School in Northport, Alabama, came up with the name Ingenuity for NASA\'s Mars Helicopter (an artist\'s impression of which is seen here) and the motivation behind it for NASA\'s "Name the Rover" essay contest. Credits: NASA/JPL-Caltech/NIA/Rupani Family. Download image ›Vaneeza Rupani\'s essay was chosen as the name for the small spacecraft, which will mark NASA\'s first attempt at powered flight on another planet.Destined to become the first aircraft to attempt powered flight on another planet, NASA\'s Mars Helicopter officially has received a new name: Ingenuity.Vaneeza Rupani, a junior at Tuscaloosa County High School in Northport, Alabama, came up with the name and the motivation behind it during NASA\'s "Name the Rover" essay contest."The ingenuity and brilliance of people working hard to

{'news_title': "Mars Helicopter Attached to NASA's Perseverance Rover", 'news_p': 'The team also fueled the rover\'s sky crane to get ready for this summer\'s history-making launch.With the launch period of NASA\'s Mars 2020 Perseverance rover opening in 14 weeks, final preparations of the spacecraft continue at the Kennedy Space Center in Florida. In the past week, the assembly, test and launch operations team completed important milestones, fueling the descent stage — also known as the sky crane — and attaching the Mars Helicopter, which will be the first aircraft in history to attempt power-controlled flight on another planet.Over the weekend, 884 pounds (401 kilograms) of hydrazine monopropellant were loaded into the descent stage\'s four fuel tanks. As the aeroshell containing the descent stage and rover enter the Martian atmosphere on Feb. 18, 2021, the propellant will be pressure-fed through 120 feet (37 meters) of stainless steel and titanium tubing into eight Mars landing engi

In [ ]:
#for url in url_list:
#        browser.visit(url)
 #       try:
#            news_title = link.find('div', 'article_title')
            #news_p = link.find('div', 'wysiwyg_content')
#            if (news_title and news_p):
#                print ('-------------')
#                print (news_title)
#                print (news_p)
        # Dictionary to be inserted as a MongoDB document
#                post = {
 #                   'news_title': news_title.strip(),
#                    'news_p': news_p.strip()
#                }
#                stories.insert_one(post)
 #               print(post)
#
#        except Exception as e:
            print(e)
#
#browser.quit()        
#print(stories)


In [54]:
NASA_news = db.stories.find()

for article in NASA_news:
    print(article)

{'_id': ObjectId('5f57063a3f49cf80eb96d715'), 'news_title': "NASA Readies Perseverance Mars Rover's Earthly Twin", 'news_p': "Test Rover Moves to Mars Yard: Technicians move an engineering version of the Perseverance Mars rover into to its new home in the Mars Yard, part of NASA's Jet Propulsion Laboratory in Southern California. Full image & caption ›"}
{'_id': ObjectId('5f5706403f49cf80eb96d716'), 'news_title': 'NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities', 'news_p': "Mars 2020 Rover Is Roving: In a clean room at NASA's Jet Propulsion Laboratory in Pasadena, California, engineers observed the first driving test for NASA's Mars 2020 rover on Dec. 17, 2019. Credits: NASA/JPL-Caltech. Full image and caption ›"}
{'_id': ObjectId('5f5706423f49cf80eb96d717'), 'news_title': "The Launch Is Approaching for NASA's Next Mars Rover, Perseverance", 'news_p': "Mars 2020 Rover Is Roving: In a clean room at NASA's Jet Propulsion Laboratory in Southern California, engineers 

### JPL Mars Space Images - Featured Image

In [ ]:
# Thanks to GitHub for the fancybox help, I was really stuck. Didn't realize "fancybox" could be a class. 
#https://github.com/ZL14E161110/HW13---Web-Scraping-and-Document-Databases
#Didn't look at the rest of it, although I'm sure it's brilliant. 

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'lxml')

#Assuming the latest Mars image will always be the one in the first position of the grid...
#Note 0 is the overall featured image, which is not necessarily of Mars.

partial_address = soup.find_all('a', class_='fancybox')[1].get('data-fancybox-href').strip()
base = "https://www.jpl.nasa.gov"
featured_image_url = urllib.parse.urljoin(base, partial_address)
featured_image_url
browser.quit()

### Mars Facts

In [ ]:
url = 'https://space-facts.com/mars/'

In [ ]:
tables = pd.read_html(url)
type(tables)

In [ ]:
tables[0]

In [ ]:
mars_facts_df = tables[0]
mars_facts_df.set_index([0], inplace=True)
mars_facts_df

In [ ]:
mars_facts_df.rename(columns=mars_facts_df.iloc[0]).drop(mars_facts_df.index[0])
mars_facts_df

In [ ]:
mars_facts_html_table = mars_facts_df.to_html()
mars_facts_html_table = mars_facts_html_table.replace('\n', '')
mars_facts_html_table
print(mars_facts_html_table)

In [ ]:
###Make sure extra header row is gone###

### Mars Hemispheres

In [ ]:
#navigate to starting page
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [ ]:
#Make soup
# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')
#print(soup.prettify)

In [ ]:
#Find titles and links for full-scale images

title_list = []
img_url_list = []
base = "https://astrogeology.usgs.gov"

for link in soup.find_all(class_='itemLink product-item'):
    partial = link.get('href')    
    thumbnail_url = urllib.parse.urljoin(base, partial)
    browser.visit(thumbnail_url)
    xpath = "//a[@class='open-toggle']"
    switch = browser.find_by_xpath(xpath)
    switch.click()
    html = browser.html
    soup = bs(html, 'lxml')
    imgs = soup.find(class_='downloads')
    html = browser.html
    soup = bs(html, 'lxml')
    title = soup.find(class_ = 'title').get_text()
    img_url = [a['href'] for a in soup.select('a:contains(Original)')]
    title_list.append(title)
    img_url_list.append(*img_url)
    print(title, *img_url)
browser.quit()
hemisphere_image_urls = zip(title_list, img_url_list)

## Step 2 - MongoDB and Flask Application

Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

* Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

* Next, create a route called `/scrape` that will call your `scrape_mars.scrape` function. Note that you'll have to import `scrape_mars.py`. 

  * Store the dictionary that gets returned to your MongoDB.

* Create an index route `/` that will query your Mongo database and pass the Mars data into an HTML template to be displayed. 

* Create a template HTML file called `index.html` that will take the dictionary of Mars data and display its values in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

![final_app_part1.png](Images/final_app_part1.png)
![final_app_part2.png](Images/final_app_part2.png)